In [28]:
# py3 网络爬虫开发实战
# 2.1 urllib 库的使用
from urllib import parse, request, error, robotparser
import socket
import http.cookiejar

GURL = 'http://httpbin.org/get'
PURL = 'http://httpbin.org/post'
BAIDU = 'https://www.baidu.com'

In [25]:
# urlopen 的使用
# request.urlopen(url, data=None, [timeout, ]*, cafile=None, capath=None, cadefault=False, context=None)
# 适用于简单请求, 无法设置复杂参数
try:
    # 可选参数 timeout: 设置超时不响应
    response = request.urlopen(GURL) #, timeout=0.1)
    print(type(response))
    print(response.status)
    print(response.getheaders())
    print(response.getheader('server'))
    print(response.read().decode('utf-8'))
except error.URLError as e:
    # error.reason 不一定是字符串, 也可能是某个实例
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT!!!')
        
# 可选参数 data: 用于 post 请求
# 需要先转化为字节流编码格式
data = bytes(parse.urlencode({'name': 'python'}), encoding='utf-8')
response = request.urlopen(PURL, data=data)
print(response.read().decode('utf-8'))

<class 'http.client.HTTPResponse'>
200
[('Date', 'Thu, 12 May 2022 09:36:25 GMT'), ('Content-Type', 'application/json'), ('Content-Length', '275'), ('Connection', 'close'), ('Server', 'gunicorn/19.9.0'), ('Access-Control-Allow-Origin', '*'), ('Access-Control-Allow-Credentials', 'true')]
gunicorn/19.9.0
{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.10", 
    "X-Amzn-Trace-Id": "Root=1-627cd519-0ea8c68d7c859b72382eb96f"
  }, 
  "origin": "223.73.53.159", 
  "url": "http://httpbin.org/get"
}

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "python"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.10", 
    "X-Amzn-Trace-Id": "Root=1-627cd51a-54f5384b06876c3233dbddae"
  }, 
  "json": null, 
  "origin": "223.73.53.159", 
  "url"

In [26]:
# Request 类的使用
# class request.Request(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)
# - data: 字节流编码格式 bytes(), 如果是字典数据, 还需要 urlencode
# - headers: 请求头字典, 可以直接构建, 也可以用 Request 实例的 add_header() 方法
# - origin_req_host: 请求方的 ip 或 host
# - unverifiable: 请求是否是无法验证的??? TODO
# - method: GET POST PUT 等

# 1. 构建 Request 类
# 创建 headers 字典
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15',
    'Host': 'www.httpbin.org',
}
data = bytes(parse.urlencode({'name': 'python'}), encoding='utf-8')

result = request.Request(url=PURL, data=data, headers=headers, method='POST')

# 继续添加 headers
result.add_header('Connection', 'keep-alive')

# 2. 依然使用 urlopen 发送请求
response = request.urlopen(result)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "python"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "www.httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15", 
    "X-Amzn-Trace-Id": "Root=1-627cd51d-1e57bb632aed3bb56d682949"
  }, 
  "json": null, 
  "origin": "223.73.53.159", 
  "url": "http://www.httpbin.org/post"
}



In [27]:
# Handler 类和 Opener 类
# 请求基本身份验证的网站

WRONG_URL = 'https://ssr3.scrape.center/404'
AURL = 'https://ssr3.scrape.center'
username = 'admin'
password = 'admin'

p = request.HTTPPasswordMgrWithDefaultRealm()
p.add_password(None, AURL, username, password)
auth_handler = request.HTTPBasicAuthHandler(p)
opener = request.build_opener(auth_handler)

try:
    result = opener.open(AURL)
    # result = opener.open(WRONG_URL) # 错误处理测试
    html = result.read().decode('utf-8')
    print(html)
except error.HTTPError as e:
    # HTTPError 是 URLError 的子类, 要优先处理
    print(e.reason, e.code, e.headers, sep='\n')
except error.URLError as e:
    print(e.reason)
else:
    print('query done!!!')


<html lang="en">
<head>
  
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <link rel="icon" href="/static/img/favicon.ico">
  <title>Scrape | Movie</title>
  

  <link href="/static/css/app.css" type="text/css" rel="stylesheet">
  
<link href="/static/css/index.css" type="text/css" rel="stylesheet">

</head>
<body>
<div id="app">
  <div data-v-74e8b908="" class="el-row" id="header">
    <div data-v-74e8b908="" class="container el-col el-col-18 el-col-offset-3">
      <div data-v-74e8b908="" class="el-row">
        <div data-v-74e8b908="" class="logo el-col el-col-4">
          <a data-v-74e8b908="" href="/" class="router-link-exact-active router-link-active">
            <img data-v-74e8b908="" src="/static/img/logo.png" class="logo-image">
            <span data-v-74e8b908="" class="logo-title">Scrape</span>
          </a>
        </div>
      </div>
    </div>
  </div>
  
<div dat

In [19]:
# cookie 处理
cookie = http.cookiejar.CookieJar()
handler = request.HTTPCookieProcessor(cookie)
opener = request.build_opener(handler)
response = opener.open(BAIDU)
for item in cookie:
    print(item.name + ': ' + item.value)

BAIDUID: 701AAE496F6E3E1D561D298DDD6507A6:FG=1
BIDUPSID: 701AAE496F6E3E1D1EFE2036BCFA9C9B
PSTM: 1652332878
BD_NOT_HTTPS: 1


In [29]:
# robots 协议
# - User-agent:
# - Disallow:
# - Allow:
rp = robotparser.RobotFileParser()
rp.set_url(BAIDU + '/robots.txt')
rp.read()
print(rp.can_fetch('Baiduspider', BAIDU))
print(rp.can_fetch('Googlebot', BAIDU + '/homepage/'))
print(rp.can_fetch('Baiduspider', BAIDU + '/homepage/'))

True
False
True


In [44]:
# parse 解析
# urllib.parse.urlparse(urlstring, scheme='', allow_fragments=True)
# - urlstring: 目标域名
# - scheme: 协议类型, 如果 urlstring 中带有, 则忽略, 默认 http Or https
# - allow_fragments: 是否单独解析 fragments
result = parse.urlparse(BAIDU + '/index.html;user?id=88#comment', scheme = 'http', allow_fragments = False)
# result 是一个元组, 可以用属性名获取, 也可以使用索引
print(result)
print(result.scheme, result[0], result.query, result[4], sep = '\n')

# urlunparse 构造 url
data = ['http', 'www.baidu.com', 'index.html', 'user', 'name=python', 'comment']
print(parse.urlunparse(data))

# urlsplit 类似 urlparse, 但不单独解析 params, params 合并到 path 中
# urllib.parse.urlsplit(urlstring, scheme='', allow_fragments=True)
result = parse.urlsplit(BAIDU + '/index.html;user?id=88#comment', scheme = 'http', allow_fragments = False)
print(result)
print(result.scheme, result[0], result.query, result[3], sep = '\n')

# urlunsplit 构造 url, 类似 urlunparse
data = ['http', 'www.baidu.com', 'index.html;user', 'name=python', 'comment']
print(parse.urlunsplit(data))

# urljoin 拼接
# urllib.parse.urljoin(base, url, allow_fragments=True)
# 分析 base 里的 scheme, netloc, path, 如果 url 中不存在这3项, 则予以补充, 如果存在, 则直接使用 url

# urlencode 将字典数据序列化为 url 参数
params = {
    'name': 'python',
    'age': 32
}
print(BAIDU + '?' + parse.urlencode(params))

# parse_qs 反序列化为字典
print(parse.parse_qs('name=python&age=32'))
# parse_qsl 反序列化为元组列表, 每一个元素为一个元组
print(parse.parse_qsl('name=python&age=32'))

# quote 对 url 中的非 ascii 字符进行编码
word = '😭'
print(parse.quote(word))
print(parse.unquote('%F0%9F%98%AD'))

ParseResult(scheme='https', netloc='www.baidu.com', path='/index.html', params='user', query='id=88#comment', fragment='')
https
https
id=88#comment
id=88#comment
http://www.baidu.com/index.html;user?name=python#comment
SplitResult(scheme='https', netloc='www.baidu.com', path='/index.html;user', query='id=88#comment', fragment='')
https
https
id=88#comment
id=88#comment
http://www.baidu.com/index.html;user?name=python#comment
https://www.baidu.com?name=python&age=32
{'name': ['python'], 'age': ['32']}
[('name', 'python'), ('age', '32')]
%F0%9F%98%AD
😭
